In [1]:
"""
County level analysis on state connection to innovation
"""

'\nCounty level analysis on state connection to innovation\n'

In [9]:
import pandas as pd
import os
import numpy as np
from fuzzywuzzy import fuzz
import numpy as np
import itertools
import networkx as nx
os.chdir("C:/Users/Joe's PC/Documents/IDEA Masters/Second Year/First Semester/Applied Public Economics/Term Paper/Patent Data/County/No Citations/New York")

In [3]:
"""
assignee = pd.read_csv("assignee.csv")

#Takes a long time!

assignee_dict = {}
for idx in assignee.index:
    key = assignee.iloc[idx][0]
    value = assignee.iloc[idx][4]
    assignee_dict.update({key : value})

np.save('assignee_dict.npy', assignee_dict)
"""

'\nassignee = pd.read_csv("assignee.csv")\n\n#Takes a long time!\n\nassignee_dict = {}\nfor idx in assignee.index:\n    key = assignee.iloc[idx][0]\n    value = assignee.iloc[idx][4]\n    assignee_dict.update({key : value})\n\nnp.save(\'assignee_dict.npy\', assignee_dict)\n'

In [4]:
assignee_dict = np.load('assignee_dict.npy',allow_pickle='TRUE').item()

In [5]:
def CleanStateData(data, file_name=""):

    data["assignee_id"] = data["assignee_id"].map(assignee_dict)
    data = data.dropna()


    for ass in data["assignee_id"].unique():
        str1 = ass
        for ass2 in data["assignee_id"].unique():
            str2 = ass2

            ratio = fuzz.ratio(str1.lower(), str2.lower())
            if 90 < ratio < 100:
                data["assignee_id"] = data["assignee_id"].replace(str2, str1, regex=True)


    data = data.sort_values("assignee_id")

    GovtInterest = data[data["govint_org_id"] != "None"]

    data.to_csv(f"{file_name}(Clean).csv")

    return data


In [6]:
def CreateGraph(data, file_name=""):

    G = data["govint_org_id"].unique()
    G = G[1:]


    govt = {"None" : 0}

    govt.update(dict.fromkeys(G, 1))

    print(govt)


    data["Govt Support"] = data["govint_org_id"].map(govt)

    print(data["Govt Support"].value_counts(normalize=True))


    data = data.sort_values("inventor_id")

    ### Create the graph ###
    state_edges = []

    for assignee in data["assignee_id"].unique():
        A = data[data["assignee_id"] == assignee]

        if 1 in A["Govt Support"].values:
            state_edges.append(("state", assignee))


    ### Add the edges ###
    edges = []

    for inventor in data["inventor_id"].unique():

        group = data[data["inventor_id"] == inventor]

        assignees = group["assignee_id"].unique()
        combinations = list(itertools.combinations(assignees, 2))

        for edge in combinations:

            edges.append(edge)

    nodes = data["assignee_id"].unique()
    nodes = np.insert(nodes, 0, "state")

    Graph = nx.Graph()

    Graph.add_nodes_from(nodes)
    Graph.add_edges_from(state_edges)
    Graph.add_edges_from(edges)

    nx.write_graphml(Graph, f"{file_name}.graphml")

    return print(nx.info(Graph))



In [20]:
file = "New York 2012"

In [21]:
data = pd.read_csv(f"{file}.csv")
clean_data = CleanStateData(data=data, file_name=f"{file}")
Graph = CreateGraph(clean_data, file_name=f"{file}")

<ipython-input-5-d7c5ead35c12>:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data["assignee_id"] = data["assignee_id"].replace(str2, str1, regex=True)


{'None': 0, '95': 1, '10': 1, '127': 1, '64': 1, '55': 1, '51': 1, '18': 1, '5': 1, '126': 1, '31': 1, '133': 1, '125': 1, '19': 1, '54': 1, '53': 1, '119': 1, '113': 1, '128': 1, '131': 1, '123': 1, '135': 1, '120': 1, '116': 1, '179': 1}
0    0.920936
1    0.079064
Name: Govt Support, dtype: float64
Name: 
Type: Graph
Number of nodes: 466
Number of edges: 87
Average degree:   0.3734
